In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

import os
from keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np

landmarks = [
    'PCLOrigin',
    'lateralCondyle',
    'medialCondyle',
    'medialSulcus',
    'whitesideReference'
]


def one_axis_femur_model(name='PCLOrigin', axis=0):
    output_path = "/code/" + name + "/"

    data_size = 15
    imgs_to_process = []
    lbls_to_process = []
    dif_arr = [(0,1,2), (0,2,1), (1,0,2), (1,2,0), (2,0,1), (2,1,0)]
    num_classes = 120

    for index in range(data_size):
        for a, b, c in dif_arr:
            imgs_to_process.append(np.load(output_path + 'images/' + '{}-{}-{}-{}.npy'.format(index, a, b, c)))
            label = np.load(output_path + 'label/' + '{}-{}-{}-{}.npy'.format(index, a, b, c))
            lbls_to_process.append(int(label[3+axis]))
            num_classes = int(label[6]) * 2

    imgs_to_process = np.array(imgs_to_process)
    lbls_to_process = np.array(lbls_to_process)

    batch_size = 15
    epochs = 200
    learning_rate = 0.001

    x_train, x_test, y_train, y_test = train_test_split(imgs_to_process, lbls_to_process, test_size=0.2)
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    colors = x_train.shape[3]


    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1,colors, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1,colors, img_rows, img_cols)
        input_shape = (1, colors, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, colors, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, colors, 1)
        input_shape = (img_rows, img_cols, colors, 1)

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    model = load_model('one_axis_femur_model_{}_{}.h5'.format(name, axis))

    score = model.evaluate(x_test, y_test, verbose=0)
    prediction = model.predict(x_test)
    for p, v in zip(prediction, y_test):
        print("prediction vs actual value: ", np.argmax(p), np.argmax(v))
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

Using TensorFlow backend.


In [2]:
one_axis_femur_model('medialSulcus', 0)

prediction vs actual value:  24 24
prediction vs actual value:  21 21
prediction vs actual value:  27 27
prediction vs actual value:  26 21
prediction vs actual value:  25 25
prediction vs actual value:  24 24
prediction vs actual value:  23 23
prediction vs actual value:  25 25
prediction vs actual value:  23 23
prediction vs actual value:  21 21
prediction vs actual value:  24 24
prediction vs actual value:  28 28
prediction vs actual value:  26 26
prediction vs actual value:  24 22
prediction vs actual value:  21 21
prediction vs actual value:  24 24
prediction vs actual value:  21 27
prediction vs actual value:  21 21
Test loss: 1.1472265720367432
Test accuracy: 0.8333333134651184


In [3]:
one_axis_femur_model('medialSulcus', 1)

prediction vs actual value:  26 26
prediction vs actual value:  27 27
prediction vs actual value:  23 23
prediction vs actual value:  24 24
prediction vs actual value:  22 22
prediction vs actual value:  28 28
prediction vs actual value:  25 25
prediction vs actual value:  21 21
prediction vs actual value:  21 21
prediction vs actual value:  24 24
prediction vs actual value:  23 23
prediction vs actual value:  24 24
prediction vs actual value:  21 21
prediction vs actual value:  23 23
prediction vs actual value:  24 24
prediction vs actual value:  24 24
prediction vs actual value:  24 24
prediction vs actual value:  22 22
Test loss: 6.901482993271202e-05
Test accuracy: 1.0


In [14]:
one_axis_femur_model('medialSulcus', 2)

prediction vs actual value:  23 23
prediction vs actual value:  25 25
prediction vs actual value:  27 27
prediction vs actual value:  27 21
prediction vs actual value:  25 25
prediction vs actual value:  24 24
prediction vs actual value:  27 27
prediction vs actual value:  21 24
prediction vs actual value:  27 25
prediction vs actual value:  24 24
prediction vs actual value:  21 26
prediction vs actual value:  21 21
prediction vs actual value:  23 23
prediction vs actual value:  28 28
prediction vs actual value:  23 23
prediction vs actual value:  24 24
prediction vs actual value:  24 24
prediction vs actual value:  23 23
Test loss: 0.5472295880317688
Test accuracy: 0.7777777910232544
